Developing Integrated Gradient Attribution. SCRATCH I kept crashing the kernel,so it might just not work on my cpu/mps

In [1]:
import torch as t
import torch.nn as nn
from torch import Tensor
from torch.autograd.functional import jacobian

import pdb

import os
from models.gpt import GPT
from models.sparsified import SparsifiedGPT, SparsifiedGPTOutput

from config.gpt.training import options
from config.sae.models import sae_options



from data.tokenizers import ASCIITokenizer, TikTokenTokenizer

from models.sae import SparseAutoencoder
from typing import Callable
from datasets import Dataset
from data.dataloaders import TrainingDataLoader
TensorFunction = Callable[[Tensor], Tensor]

/Users/andrewgordon/Documents/ML/TeamStefan/PeterLaiSAE/PLgpt-circuits/myenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
t.mps.empty_cache()
c_name = 'standardx8.shakespeare_64x4'
name = 'standard.shakespeare_64x4'
data_dir = 'data/shakespeare'
batch_size = 1
config = sae_options[c_name]


model = SparsifiedGPT(config)
model_path = os.path.join("checkpoints", name)
model = model.load(model_path, device=config.device)

model.to(device)  # Move model


ddp = int(os.environ.get("RANK", -1)) != -1  # is this a ddp run?
if ddp:
    # use of DDP atm demands CUDA, we set the device appropriately according to rank
    ddp_rank = int(os.environ["RANK"])
    ddp_local_rank = int(os.environ["LOCAL_RANK"])
    ddp_world_size = int(os.environ["WORLD_SIZE"])
    device = t.device(f"cuda:{ddp_local_rank}")

    assert torch.cuda.is_available()
    t.cuda.set_device(device)
else:
    # vanilla, non-DDP run
    ddp_rank = 0
    ddp_local_rank = 0
    ddp_world_size = 1
    device = config.device

dataloader = TrainingDataLoader(
    dir_path=data_dir,
    B= batch_size,
    T=model.config.block_size,
    process_rank=ddp_rank,
    num_processes=ddp_world_size,
    split="val",
)
x,y = dataloader.next_batch(device)

input, _ = dataloader.next_batch(device) #get batch of inputs
input.to(t.int32) #hacky to make thigs work on my M3, remove later






Found 1 shards for split val
cpu


In [ ]:
co

In [ ]:

output = model.forward(input, targets=None, is_eval=True)
feature_magnitudes0 = output.feature_magnitudes[0] #feature magnitudes at source layer (batchsize, seqlen, sae_encoding_dim)


In [27]:
send_to_cpu(model)

In [26]:
def send_to_cpu(model):
    device = t.device("cpu")
    for name, param in model.named_parameters():
        param.to(device)
    for saekey in model.saes:
        model.saes[saekey].to(device)

In [31]:
layer0 = 0
layer1 = 1
sae0 = model.saes[f'{layer0}']
sae1 = model.saes[f'{layer1}']


#define function that goes from feature magnitudes in layer0 to feature magnitudes in layer1
class Sae0Decode(nn.Module):
    def forward(self, x):
        return sae0.decode(x)
    
class Sae1Encode(nn.Module):
    def forward(self, x):
        return sae1.encode(x)

forward_list = [Sae0Decode()] + [model.gpt.transformer.h[i] for i in range(layer0, layer1)] + [Sae1Encode()]
forward = t.nn.Sequential(*forward_list)

In [41]:
input, _ = dataloader.next_batch(device) #get batch of inputs

input.to(t.int32) #hacky to make thigs work on my M3, remove later

output = model.forward(input, targets=None, is_eval=True)
feature_magnitudes0 = output.feature_magnitudes[layer0]


In [42]:
forward(feature_magnitudes0)

tensor([[[0.0000, 0.0095, 0.0000,  ..., 0.0022, 0.0000, 0.0000],
         [0.0000, 0.0339, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0316, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
       grad_fn=<ReluBackward0>)

In [47]:
ig_attributions(model, 0, 1, dataloader)

: 

In [46]:
def ig_attributions(model: SparsifiedGPT, layer0: int, layer1: int, ds: TrainingDataLoader, nbatches: int=32):
    assert layer0 < layer1
    assert layer0 >= 0
    assert layer1 <= model.gpt.config.n_layer
    sae0 = model.saes[f'{layer0}']
    sae1 = model.saes[f'{layer1}']
    #pdb.set_trace()

    #define function that goes from feature magnitudes in layer0 to feature magnitudes in layer1
    class Sae0Decode(nn.Module):
        def forward(self, x):
            return sae0.decode(x)
        
    class Sae1Encode(nn.Module):
        def forward(self, x):
            return sae1.encode(x)
    
    forward_list = [Sae0Decode()] + [model.gpt.transformer.h[i] for i in range(layer0, layer1)] + [Sae1Encode()]
    forward = t.nn.Sequential(*forward_list)

    
    attributions = 0
    for _ in range(nbatches):

        input, _ = ds.next_batch(device) #get batch of inputs #TODO FIX THIS
        
        input.to(t.int32) #hacky to make thigs work on my M3, remove later
        output = model.forward(input, targets=None, is_eval=True)
        feature_magnitudes0 = output.feature_magnitudes[layer0] #feature magnitudes at source layer (batchsize, seqlen, sae_encoding_dim)
        
        
        jacobian = integrate_gradient(feature_magnitudes0, None, forward) #(batch seq dim batch seq dim)
        attributions = attributions + (jacobian **2).sum(dim=[0,1,3,4]) #Does this make sense? It feels incorrect to sum over position
            #maybe a better output would have shape (seqlen, dim, seqlen, dim)?
        


    return attributions


In [36]:
def integrate_gradient(x: Tensor, x_i: Tensor | None, y: TensorFunction, base:int= 0, steps:int=10):
    """
    Approximates int_C d/dx_i y(z) dz where C is a linear path from base to x
    :param x: End of path. It can be either a vector of feature magnitudes generated by the data, 
        or a one hot encoding of a feature magnitude. It should have a nonzero entry for x_i (???)
        Shape (batchsize, seq_len, encoding)
    :param x_i: Direction of partial derivative. It is often a one hot encoding of a given feature magnitude. 
        If none, this computes the entire Jacobian
        Shape (batchzise, seq_len, encoding) ??Support just encoding??
    :param y: Tensor valued function on the vector space containing x
    :param base: Start of path. Default to 0
    :param steps: Number of steps to use to approximate the integral
    :return: Tensor of shape ((output of y), x.shape) if x_i = None else shape (output of y)

    
    """
    
    if base == 0:
        base = t.zeros_like(x)
    path = t.linspace(0, 1, steps)
    steplength = t.linalg.norm(x - base, dim = -1, keepdim = True)/steps

    integral = 0
    for alpha in path:
        point = (alpha*x + (1-alpha)*base).detach() #Find point on path
        jac = jacobian(y, point)*steplength #compute jacobian of y wrt x, scale by length of a step
        
        if x_i != None:
            n = len(x.shape)
            backdims = list(range(-1*n, 0))
            integral += (jac * x_i).sum(dim = backdims)#Evaluate Jacobian in x_i direction
        else:
            integral += jac


    return integral


In [55]:
x = t.Tensor([[1,2],[3,4]])
x_i = None
r = t.randn_like(x)

y = lambda z: r@z
j = integrate_gradient(x,x_i, y, steps = 20)
steps = 20


n = len(x.shape)
backdims = list(range(-1*n, 0))
x_i = t.zeros_like(x)
x_i[0,0] = 1

print((j * x_i).sum(dim = backdims)) #Evaluate Jacobian in x_i direction
print(integrate_gradient(x,x_i, y, steps = 20))



tensor([[1.4995, 0.0000],
        [1.1770, 0.0000]])
tensor([[1.4995, 0.0000],
        [1.1770, 0.0000]])


In [8]:
list(range(-10,0))

[-10, -9, -8, -7, -6, -5, -4, -3, -2, -1]

In [53]:
def sum(a,b,c):
    return a + b + c
sum(*[1,2,3])

6

In [2]:
c_name = 'standardx8.shakespeare_64x4'
name = 'standard.shakespeare_64x4'
data_dir = 'data/shakespeare'
batch_size = 1
config = sae_options[c_name]
config.device

device(type='mps')